In [1]:
# Importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from math import ceil

In [2]:
#Reading data files
trips = {}
trips['trip1_acc'] = pd.read_csv("data/1/Accelerometer.csv")
trips['trip1_lac'] = pd.read_csv("data/1/Linear_Acceleration.csv")
trips['trip1_gyr'] = pd.read_csv("data/1/Gyrometer.csv")
trips['trip1_mag'] = pd.read_csv("data/1/Magnetometer.csv")
trips['trip1_lbl'] = pd.read_csv("data/1/Truth.csv")

trips['trip2_acc'] = pd.read_csv("data/2/Accelerometer.csv")
trips['trip2_lac'] = pd.read_csv("data/2/Linear_Acceleration.csv")
trips['trip2_gyr'] = pd.read_csv("data/2/Gyrometer.csv")
trips['trip2_mag'] = pd.read_csv("data/2/Magnetometer.csv")
trips['trip2_lbl'] = pd.read_csv("data/2/Truth.csv")

trips['trip3_acc'] = pd.read_csv("data/3/Accelerometer.csv")
trips['trip3_lac'] = pd.read_csv("data/3/Linear_Acceleration.csv")
trips['trip3_gyr'] = pd.read_csv("data/3/Gyrometer.csv")
trips['trip3_mag'] = pd.read_csv("data/3/Magnetometer.csv")
trips['trip3_lbl'] = pd.read_csv("data/3/Truth.csv")

trips['trip4_acc'] = pd.read_csv("data/4/Accelerometer.csv")
trips['trip4_lac'] = pd.read_csv("data/4/Linear_Acceleration.csv")
trips['trip4_gyr'] = pd.read_csv("data/4/Gyrometer.csv")
trips['trip4_mag'] = pd.read_csv("data/4/Magnetometer.csv")
trips['trip4_lbl'] = pd.read_csv("data/4/Truth.csv")

In [3]:
# Irrelevant timestamp column dropping
# Also, determine smallest and largest window sizes for the events
min_window = 1000000
max_window = 0

for name,trip in trips.items():
    if 'timestamp' in trip.columns:
        trip.drop(['timestamp'], axis=1, inplace=True)
    
    if 'uptimeNanos' in trip.columns:
        uptimeNanos = np.array(trip['uptimeNanos'])
        timeInSec = (uptimeNanos - uptimeNanos[0])/1000000000
        trip['timeInSec'] = timeInSec
        trip.drop(['uptimeNanos'], axis=1, inplace=True)
        
    if 'event' in trip.columns:
        maxW = max(trip.finish - trip.start)
        minW = min(trip.finish - trip.start)
        
        if maxW > max_window:
            max_window = maxW
        if minW < min_window:
            min_window = minW

In [4]:
# Rounding of the window size to the ceil of the time difference between start and finish of the event 
print("Minimum event duration: ", min_window, "s   Minimum Window Size: " , ceil(min_window), "s")
min_window = ceil(min_window)
print("Maximum event duration: ", max_window, "s   Maximum Window Size: " , ceil(max_window), "s")
max_window = ceil(max_window)

Minimum event duration:  1.5999999999999979 s   Minimum Window Size:  2 s
Maximum event duration:  4.899999999999977 s   Maximum Window Size:  5 s


In [5]:
# Storing the indices of 1sec frames 
allTripPerSecIndex = {}
for name,trip in trips.items():
    if 'event' not in trip.columns:
        tmpFrame = pd.DataFrame(columns=['perSecIndex'])
        count = 0
        for i in range(0,len(trip)-1,1):
            if trip['timeInSec'][i]-count > 1:
                tmpFrame = tmpFrame.append({'perSecIndex':i-1},ignore_index=True)
                count = count + 1
        allTripPerSecIndex[name+'_perSecIndex'] = tmpFrame

In [6]:
# Function getFeatureNames
# Input:
#     s = sensor_name
#     w = window_size
# Output:
#     l = [feature names as object, feature name as 

# params = [Mean, Median, Std Deviation, Mean Abs Deviation, Min, Max, Tendency]
def getFeatureNames( s, w ):
    l = []
    c = []
    d = {}
    params = ['mean','median','sd','td']
    
    for i in range(1,w+1):
        for j in range(0,len(params),1):
            v1 = s + '_' + params[j] + '_x' + str(i)
            v2 = s + '_' + params[j] + '_y' + str(i)
            v3 = s + '_' + params[j] + '_z' + str(i)
            
            if params[j] != 'td':
                c.append(v1)
                c.append(v2)
                c.append(v3)
                
                d[v1] = 0.0
                d[v2] = 0.0
                d[v3] = 0.0
            else:
                if i != w:                    
                    c.append(v1)
                    c.append(v2)
                    c.append(v3)
                    
                    d[v1] = 0.0
                    d[v2] = 0.0
                    d[v3] = 0.0
    l.append(c)
    l.append(d)
    
    return l

In [7]:
# Considering window_size = 3
w = 3
dataSensorwise = {}
for name,tripIndex in allTripPerSecIndex.items():
    i = 0
    j = w
    end = len(tripIndex)
    l = getFeatureNames(name[6:len(name)-12],w)
    tripFeature = pd.DataFrame(columns=l[0])
    tripRawData = trips[name[:len(name)-12]]
    
    featureVector = l[1]   
    key_mean = name[6:len(name)-12] + '_mean_'
    key_median = name[6:len(name)-12] + '_median_'
    key_sd = name[6:len(name)-12] + '_sd_'
    key_td = name[6:len(name)-12] + '_td_'
    setLastInd = 0
    
    while(j<=end):    
        lastInd = setLastInd
        frameCount = 1
        
        for k in range(i,j,1):            
            ind = tripIndex.perSecIndex.values[k]
            
            featureVector[key_mean + 'x' + str(frameCount)] = tripRawData.x[lastInd:ind+1].mean()
            featureVector[key_mean + 'y' + str(frameCount)] = tripRawData.y[lastInd:ind+1].mean()
            featureVector[key_mean + 'z' + str(frameCount)] = tripRawData.z[lastInd:ind+1].mean()
            
            featureVector[key_median + 'x' + str(frameCount)] = tripRawData.x[lastInd:ind+1].median()
            featureVector[key_median + 'y' + str(frameCount)] = tripRawData.y[lastInd:ind+1].median()
            featureVector[key_median + 'z' + str(frameCount)] = tripRawData.z[lastInd:ind+1].median()
            
            featureVector[key_sd + 'x' + str(frameCount)] = tripRawData.x[lastInd:ind+1].std()
            featureVector[key_sd + 'y' + str(frameCount)] = tripRawData.y[lastInd:ind+1].std()
            featureVector[key_sd + 'z' + str(frameCount)] = tripRawData.z[lastInd:ind+1].std()                       
            
            # Set td only if it is not first frame
            if frameCount != 1:
                featureVector[key_td + 'x' + str(frameCount-1)] = featureVector[key_mean + 'x' + str(frameCount)]/featureVector[key_mean + 'x' + str(frameCount-1)]
                featureVector[key_td + 'y' + str(frameCount-1)] = featureVector[key_mean + 'y' + str(frameCount)]/featureVector[key_mean + 'y' + str(frameCount-1)]
                featureVector[key_td + 'z' + str(frameCount-1)] = featureVector[key_mean + 'z' + str(frameCount)]/featureVector[key_mean + 'z' + str(frameCount-1)]  
            else:
                setLastInd = ind;
                
            frameCount = frameCount + 1
        
        tripFeature = tripFeature.append(featureVector,ignore_index=True) 
        i = i + 1
        j = j + 1
    dataSensorwise[name[:len(name)-12]] = tripFeature       
            

In [8]:
dataSensorwise['trip1_acc']

,acc_mean_x1,acc_mean_y1,acc_mean_z1,acc_median_x1,acc_median_y1,acc_median_z1,acc_sd_x1,acc_sd_y1,acc_sd_z1,acc_td_x1,...,acc_td_z2,acc_mean_x3,acc_mean_y3,acc_mean_z3,acc_median_x3,acc_median_y3,acc_median_z3,acc_sd_x3,acc_sd_y3,acc_sd_z3
0,0.014650,0.066562,9.655590,0.034021,0.053366,9.657350,0.077052,0.168886,0.086906,3.169775,...,1.005306,0.084528,0.515474,9.727264,0.071605,0.316380,9.712666,0.110747,0.504592,0.119027
1,0.078522,0.316835,9.694672,0.079507,0.284995,9.698062,0.054473,0.171353,0.060777,1.522992,...,1.003849,0.105034,0.641496,9.799727,0.107950,0.517394,9.759155,0.156547,0.444750,0.149842
2,0.159810,1.165243,9.829725,0.147632,1.191789,9.835896,0.130797,0.175204,0.120871,0.739213,...,0.998614,0.087777,0.496236,9.838789,0.126864,0.501607,9.851154,0.238663,0.611476,0.174041
3,0.082479,0.454344,9.876417,0.128526,0.501607,9.913660,0.226395,0.341232,0.176581,0.663747,...,0.994622,0.034867,-0.055496,9.791187,0.045645,-0.059376,9.797557,0.272407,0.547176,0.215558
4,0.029139,-0.125489,9.816298,0.103195,-0.127664,9.821164,0.310038,0.367453,0.210378,0.415150,...,0.996669,0.028088,-0.290454,9.718440,0.009272,-0.262158,9.744424,0.366660,0.538728,0.341297
5,-0.001085,-0.496251,9.688715,-0.003167,-0.464454,9.702961,0.269313,0.423928,0.217006,-27.053360,...,1.001393,0.033726,-0.276324,9.684429,0.024531,-0.272126,9.696293,0.439796,0.565956,0.385110
6,0.071962,-0.249801,9.664024,0.001977,-0.262157,9.689132,0.490301,0.695298,0.500399,0.797721,...,0.999613,0.053349,-0.115537,9.683858,0.079218,-0.136851,9.698544,0.484962,0.576266,0.424675
7,0.050444,-0.076104,9.697041,0.132796,-0.104480,9.695039,0.524894,0.458806,0.404069,0.948102,...,1.002120,0.033051,-0.013381,9.707306,0.087594,-0.008097,9.730220,0.474703,0.489015,0.375060
8,0.039873,-0.011531,9.665081,0.055025,0.047058,9.699568,0.440229,0.523039,0.379176,0.544725,...,0.999735,0.025852,-0.009604,9.703952,0.044085,0.003898,9.742777,0.553960,0.532489,0.401673
9,-0.001839,0.061352,9.754614,0.010382,0.028062,9.777472,0.456232,0.480796,0.347102,-8.704333,...,0.997019,0.003783,-0.045717,9.697963,0.014704,-0.097625,9.753572,0.734289,0.601669,0.436873


In [12]:
trips['trip1_lbl']

,event,start,finish
0,Non-Aggressive,2.0,6.5
1,Aggressive Right Curve,19.5,23.5
2,Non-Aggressive,30.0,33.5
3,Aggressive Right Curve,95.0,98.0
4,Aggressive Left Turn,247.0,251.5
5,Aggressive Left Turn,348.7,352.3
6,Non-Aggressive,485.0,489.0
7,Aggressive Left Turn,496.0,499.5
8,Aggressive Right Curve,587.0,590.0
9,Aggressive Left Turn,750.0,753.8
